## Traffic Rank

### Viewing Chicago Taxi Rides 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the data from the downloaded file
taxi_data = pd.read_csv('chicago-taxi-rides.csv')

# Apply logarithmic transformation to the number of trips for better visualization
taxi_data['log_trips'] = np.log(taxi_data['trips'].replace(0, 1))  # log to handle zero values

# Check the actual range of log_trips
print(f"Min log_trips: {taxi_data['log_trips'].min()}")
print(f"Max log_trips: {taxi_data['log_trips'].max()}")

# Normalize the hue values based on the actual range of log_trips
vmin = 0  # Set this based on your expected minimum value (adjust if necessary)
vmax = taxi_data['log_trips'].max()  # Set this to the actual max log_trips value
norm = plt.Normalize(vmin=vmin, vmax=6)  # Adjust vmax based on the maximum log value you want to display

# Create scatter plot
plt.figure(figsize=(10, 8))
scatter = sns.scatterplot(x='dropoff_community_area', y='pickup_community_area', hue='log_trips',
                          palette='Purples', data=taxi_data, hue_norm=norm)

# Customize the colorbar with specific ticks based on the expected log range
cbar = plt.colorbar(scatter.collections[0], ticks=[0.0, 1.5, 3.0, 4.5, 6.0])
cbar.set_label('log_trips')

# Add labels and title
plt.xlabel('Dropoff Community Area')
plt.ylabel('Pickup Community Area')
plt.title('Chicago Taxi Rides Scatter Plot (Log Trips)')

# Show plot
plt.show()


[15 points] The given data has rows (i, j, Tij), sometimes known as Coordinate format. It is supported by many libraries, SciPy.sparse.coo_matrix among them. Since the matrix is just 77 × 77, a “big data” technique is not required here. Read the data as a matrix.

In [ ]:
import pandas as pd
from scipy.sparse import coo_matrix

# Load the data from the CSV (replace with your file path)
taxi_data = pd.read_csv('chicago-taxi-rides.csv')

# Filter rows with valid community area numbers (0-76)
taxi_data = taxi_data[(taxi_data['pickup_community_area'] <= 76) & (taxi_data['dropoff_community_area'] <= 76)]

# Number of community areas (77)
n_areas = 77

# Create a sparse matrix for the traffic data
rows = taxi_data['pickup_community_area']
cols = taxi_data['dropoff_community_area']
data = taxi_data['trips']

# Build the matrix
traffic_matrix = coo_matrix((data, (rows, cols)), shape=(n_areas, n_areas)).toarray()

# Convert to DataFrame for viewing
df_traffic_matrix = pd.DataFrame(traffic_matrix)

# Set Pandas options for scrolling large outputs
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the matrix
df_traffic_matrix


[15 points] Using your formulation of the TrafficRank algorithm, calculate the rankings of the Chicago community areas after 0, 1, 2, 3, 4, 5 and 6 iterations of the algorithm. 

In [ ]:
import numpy as np

# Define the decaying factor
beta = 0.85
n = traffic_matrix.shape[0]

# Initialize rank values equally
ranks = np.ones(n) / n

# TrafficRank calculation for 6 iterations
for iteration in range(7):  # 0 to 6 iterations
    new_ranks = np.zeros(n)
    
    # Iterate through the matrix to calculate new rank values
    for i in range(n):
        column_sum = traffic_matrix[:, i].sum()
        
        # Avoid division by zero by skipping or adjusting for columns with zero sum
        if column_sum != 0:
            new_ranks += beta * (traffic_matrix[:, i] / column_sum) * ranks[i]
        else:
            # If column sum is zero, we skip it or distribute its rank equally (random surfer)
            new_ranks += 0

    # Handle the random surfer component (1 - beta)
    new_ranks += (1 - beta) / n
    
    # Update ranks
    ranks = new_ranks
    
    # Output the ranks after each iteration
    print(f"Ranks after iteration {iteration}:")
    print(ranks)


[15 points] An alternate measure of economic rank of an area might be the inverse of a “hardship index,” defined and quantified here. How well, or poorly, do your calculations of TrafficRank for Chicago community areas correspond with the inverse of hardship index? Give a qualitative analysis in plain English.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression  # Import LinearRegression


# TrafficRank results after 6 iterations (replace with your actual final TrafficRank values)
traffic_rank = np.array([
    0.00194805, 0.0048037, 0.00521148, 0.01409944, 0.00578656, 0.00687414,
    0.05164961, 0.04236718, 0.26667495, 0.00204635, 0.00286565, 0.00478151,
    0.00240562, 0.0029054, 0.00440995, 0.00317413, 0.00579602, 0.00245707,
    0.00222416, 0.0025596, 0.00222682, 0.00463728, 0.01130025, 0.00254804,
    0.02974212, 0.0028443, 0.00226055, 0.00243308, 0.07855548, 0.00231808,
    0.00251838, 0.00323035, 0.18910124, 0.02340024, 0.00347435, 0.00427485,
    0.00252511, 0.00254866, 0.00334522, 0.00482727, 0.00266624, 0.00994339,
    0.0035427, 0.00396086, 0.00464357, 0.00264321, 0.002696, 0.00233722,
    0.00294138, 0.0045951, 0.00257747, 0.00297277, 0.00217643, 0.0026316,
    0.0024057, 0.00233052, 0.0227573, 0.00241855, 0.00279393, 0.00549304,
    0.00266807, 0.00264336, 0.00234946, 0.00236995, 0.0025655, 0.00233998,
    0.00254356, 0.00266139, 0.00276396, 0.00339301, 0.00260952, 0.00261109,
    0.00215514, 0.0025012, 0.00206385, 0.0022758, 0.04676693
])

print("Number of ranks:", len(traffic_rank))

# Hardship Index data for the 77 community areas (replace with actual values from the document)
hardship_index = np.array([
    39.4, 47.3, 31.5, 21.7, 16.9, 9.9, 10.3, 8.6, 21.8, 28.2, 31.3, 26.7, 42.8, 45.7, 
    36.2, 32.3, 33.4, 43.9, 55.9, 54.3, 38.6, 25.6, 60.3, 18.7, 53.1, 68.3, 58.9, 26.6, 
    59.8, 70.6, 50.1, 9.0, 11.2, 63.5, 42.5, 53.2, 64.9, 49.8, 34.6, 60.2, 25.3, 50.4, 51.4, 
    47.9, 47.6, 54.9, 51.2, 38.4, 52.6, 43.3, 58.1, 56.7, 54.3, 84.2, 37.0, 38.7, 56.1, 
    66.1, 51.5, 41.9, 62.6, 49.2, 65.3, 38.8, 53.7, 59.2, 63.3, 70.5, 54.3, 40.8, 51.5, 
    24.5, 43.3, 25.6, 36.7, 33.8, 28.9
])

print("Number of hardship_index:", len(hardship_index))


# Compute the inverse of the Hardship Index
inverse_hardship = 1 / hardship_index

# Normalizing TrafficRank values (log transformation to deal with skewed data)
traffic_rank_log = np.log1p(traffic_rank)

# Create a DataFrame to compare TrafficRank and Inverse Hardship Index
df_comparison = pd.DataFrame({
    'Community Area': range(1, 78),
    'TrafficRank (log)': traffic_rank_log,
    'Inverse Hardship Index': inverse_hardship
})

# Display the comparison
print(df_comparison)

# Regression analysis to measure correlation between TrafficRank (log) and Inverse Hardship Index
X = df_comparison['TrafficRank (log)'].values.reshape(-1, 1)
y = df_comparison['Inverse Hardship Index'].values

model = LinearRegression()
model.fit(X, y)
score = model.score(X, y)
print(f"R-squared: {score}")

# Plot the comparison
plt.figure(figsize=(12, 6))
plt.scatter(df_comparison['TrafficRank (log)'], df_comparison['Inverse Hardship Index'])
plt.title('TrafficRank (Log) vs Inverse Hardship Index for Chicago Community Areas')
plt.xlabel('TrafficRank (Log)')
plt.ylabel('Inverse Hardship Index')
plt.grid(True)

# Highlight outliers with annotations
outliers = df_comparison[df_comparison['TrafficRank (log)'] > 0.05]
for _, row in outliers.iterrows():
    plt.annotate(f"CA {row['Community Area']}", (row['TrafficRank (log)'], row['Inverse Hardship Index']),
                 textcoords="offset points", xytext=(5, 5), ha='center')

plt.show()

The R-squared value of 0.1175, as shown in the analysis, points to a weak correlation between the TrafficRank of Chicago community areas and their Inverse Hardship Index. This low R-squared value means that only about 11.75% of the variation in the Inverse Hardship Index can be explained by the TrafficRank.

The visual scatter plot further supports this weak relationship. Most points cluster tightly in the lower left corner of the graph, indicating that many areas with low TrafficRank values also have low inverse hardship scores. However, there are notable outliers, such as Community Areas (CA) 7, 9, 29, and 33, which deviate significantly. These areas exhibit relatively high TrafficRank values but vary widely in terms of economic hardship.

For instance:

CA 9 has a particularly high TrafficRank (indicating more taxi traffic) but also a high inverse hardship index, which implies better economic conditions than most other areas.
CA 33 and CA 7 have higher TrafficRank values and stand out as areas where the hardship is not proportionately higher, suggesting they may be more economically prosperous despite high taxi activity.
CA 29 appears in the middle range for both TrafficRank and hardship, suggesting some alignment but not enough to significantly shift the overall correlation.
This weak correlation may be explained by several factors:

Taxi Traffic vs. Economic Activity: TrafficRank, which reflects the volume of taxi rides between community areas, might not directly capture the economic rank of these areas. High taxi traffic could be driven by factors unrelated to economic hardship, such as tourism, commercial hubs, or proximity to key transportation hubs.
Geographical and Social Differences: Certain areas might experience high traffic due to their geographical location or social functions (e.g., airports, shopping districts), but this doesn't necessarily correlate with lower economic hardship.
Inverse Hardship Index Limitations: The inverse hardship index, which is based on various socio-economic factors like poverty rates and unemployment, might not fully capture the types of mobility or demand that generate taxi rides. For instance, an economically distressed area could still have high taxi traffic if it serves as a transportation hub.
In summary, while TrafficRank gives insights into the mobility patterns in Chicago, it doesn't strongly align with economic measures like the inverse hardship index. The weak correlation suggests that taxi traffic is influenced by a broader set of factors beyond just economic hardship.

## Text Processing


### About TDF


[4 points] We can calculate TF values of each word in a given document. Explain why the calculation of IDF can only apply to a corpus, not to a specific document.


IDF applies to a corpus, not a specific document, because it measures how rare or common a term is across all documents in the corpus. It reflects the significance of a word in the entire dataset, not just in one document. A word like "the" may appear frequently in a single document, but its IDF value will be low because it appears in almost every document in the corpus.

[6 points] The implementation of Scikit-Learn’s IDF calculation differs from that of the “standard” calculation. What is the justification for this change?


Scikit-Learn's modification ensures that terms that occur in all documents still get a meaningful IDF score by avoiding division by zero and keeping IDF scores positive. This change stabilizes the calculation, especially for edge cases with extremely rare or common words.

[25 points] Each president has a .tar.gz file containing his speeches. Write a procedure to calculate TF.IDF for any president’s speeches and print the top-15 most important words in their speech. 


In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import requests

# Load stopwords dynamically
def load_stopwords():
    stopwords_url = "https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt"
    stopwords_list = requests.get(stopwords_url).content
    stopwords = set(stopwords_list.decode().splitlines())
    return list(stopwords)

# Function to clean and preprocess text
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text.lower())
    return text

# Function to calculate and print the top 15 most important words in terms of TF-IDF
def calculate_tfidf_for_speech(president, speech_file=None):
    base_dir = f"C:/Users/adity/OneDrive/Tufts University Online MS CS/2024/Fall Sem/CS 119/Quizzes/Quiz3/prez_speeches/{president}"
    
    # Check if a specific speech file is provided or process all speeches of the president
    if speech_file:
        files = [os.path.join(base_dir, speech_file)]  # Use the specific speech file
    else:
        files = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if f.endswith('.txt')]  # Use all speeches
    
    documents = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            documents.append(clean_text(text))
    
    # Initialize TfidfVectorizer
    stopwords = load_stopwords()
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    
    # Fit and transform the documents
    tfidf_matrix = vectorizer.fit_transform(documents)
    
    # Get feature names (words) and scores
    feature_names = vectorizer.get_feature_names_out()
    
    # Calculate the average TF-IDF score for each word across all documents
    avg_tfidf_scores = tfidf_matrix.mean(axis=0).A1
    word_scores = list(zip(feature_names, avg_tfidf_scores))
    
    # Sort by TF-IDF score
    word_scores_sorted = sorted(word_scores, key=lambda x: x[1], reverse=True)
    
    # Print top 15 words by TF-IDF
    print(f"Top 15 important words for {president} (speech: {speech_file or 'all'}):")
    for word, score in word_scores_sorted[:15]:
        print(f"{word}: {score:.5f}")

# Example usage:
# To calculate TF-IDF for a specific speech file of a president
calculate_tfidf_for_speech("lincoln", "lincoln_speeches_000.txt")

# To calculate TF-IDF for all speeches of a president
calculate_tfidf_for_speech("lincoln")


[15 points] Examine the result carefully – at least some of the top TF.IDF words should be historically consistent with what was going on in the country at the time. You only have a slight control over the outcome through starting with an initial set of stopwords and adding more words to the list of stopwords.

The top words identified by TF-IDF from Lincoln’s speeches clearly align with the key themes of his presidency, especially slavery, the Union, and the legal and moral battles surrounding these issues. Lincoln's rhetoric focused heavily on the preservation of the Union during the Civil War era, the abolition of slavery, and the principles of democracy and human rights, all of which are captured in the TF-IDF analysis.

The presence of words like "slavery," "states," "union," and "government" shows that the TF-IDF algorithm successfully captures the core issues of Lincoln’s time, consistent with the historical context of his speeches. Additionally, words like "constitution," "people," and "law" reflect Lincoln’s emphasis on governance and the legal framework of the United States during a period of immense crisis.

In conclusion, the important words identified by TF-IDF are historically consistent with the events and challenges of Lincoln’s presidency, reflecting his focus on the Union, the abolition of slavery, and the defense of democracy.